In [ ]:
import numpy as np
import plotly.graph_objs as go
from ipywidgets import HBox, VBox

class DataChecker:
    def __init__(self, file_path, exposure=500, gate_info=(0, 200)):
        self.file_path = file_path
        self.exposure = exposure
        self.gate_info = (gate_info[0] + 15, gate_info[1] - 10)
        self.matrix_data = None
        self.load_data()

    def load_data(self):
        with open(self.file_path, 'rb') as f:
            raw_data = np.frombuffer(f.read(), dtype=np.uint16)
        if len(raw_data) % 4096 != 0:
            raise ValueError("数据长度无法整除 4096，无法构建矩阵。")
        num_lines = len(raw_data) // 4096
        self.matrix_data = raw_data.reshape(num_lines, 4096).astype(float)

    def draw_strength(self):
        if self.exposure > self.matrix_data.shape[0]:
            raise ValueError("曝光时间超过文件大小")

        selected_data = self.matrix_data[:self.exposure, :]
        mask = (selected_data >= self.gate_info[0]) & (selected_data < self.gate_info[1])
        count_under = mask.sum(axis=0) / self.exposure
        gray_values = np.round(count_under * 255).astype(int)

        # 转置强度图
        strength_matrix = gray_values.reshape(64, 64)
        # strength_matrix = strength_matrix.T

        fig_strength = go.FigureWidget(
            data=[go.Heatmap(z=strength_matrix, colorscale='Gray', zmin=0, zmax=255)]
        )
        fig_strength.update_layout(
            title=f"强度图 - 延迟: {self.gate_info[0]}, 门宽: {self.gate_info[1] - self.gate_info[0]}",
            xaxis=dict(title="X", scaleanchor='y', scaleratio=1),
            yaxis=dict(title="Y", autorange='reversed'),
            width=512,
            height=470,
            autosize=False,
            margin=dict(l=50, r=30, t=50, b=50)
        )

        fig_hist = go.FigureWidget(data=[go.Bar(x=[], y=[])])
        fig_hist.update_layout(
            title="光子直方图",
            xaxis=dict(title="Value"),
            yaxis=dict(title="Frequency"),
            width=1024,
            height=470,
            margin=dict(l=60, r=30, t=50, b=50)
        )

        @fig_strength.data[0].on_click
        def update_hist(trace, points, state):
            if points.xs and points.ys:
                x_idx = points.xs[0]
                y_idx = points.ys[0]
                index = y_idx * 64 + x_idx
                data = self.matrix_data[:self.exposure, index]
                low, high = self.gate_info
                filtered = data[(data >= low) & (data < high)]
                hist_counts, bin_edges = np.histogram(filtered, bins=range(low, high + 1))
                fig_hist.data[0].x = bin_edges[:-1]
                fig_hist.data[0].y = hist_counts

                flux_value = (len(filtered) / self.exposure) * 100
                fig_hist.update_layout(
                    title=f"像素({x_idx}, {y_idx})的直方图\n光通量: {flux_value:.2f}%",
                    xaxis=dict(range=[low, high])
                )

        layout = HBox([fig_strength, fig_hist])
        return layout
        

    
checker = DataChecker(file_path="/Users/ming/Documents/PythonCode/Coates/src/DataChecker/2025-01-17_15-09-54_Delay-0_Width-200.raw", exposure=500, gate_info=(0, 200))
display(checker.draw_strength())


    'data': [{'colorscale': [[0.0, 'rgb(0, 0, 0)'], [0.09090909090909091, 'rgb(1…

In [1]:
# 在Jupyter笔记本中使用
from DataChecker import DataLoader
from IPython.display import display

file_path = '/Users/ming/Documents/PythonCode/Coates/src/DataChecker/2025-01-17_15-09-54_Delay-0_Width-200.raw'
data_loader = DataLoader(file_path, exposure=500, gate_info=(0, 200))
layout = data_loader.draw_strength()
display(layout)  # 这在Jupyter中可以工作

    'data': [{'colorscale': [[0.0, 'rgb(0, 0, 0)'], [0.09090909090909091, 'rgb(1…